Function decorators allow for marking a function, and potentially enhancing its behaviour.

Building your own decorators requires a good understanding of closures.

## Decorators 101

A decorator is a callable that taekes another function as an argument.

```
@decorate
def target():
    print('running target')```
    
is equivalent to

```
def target():
    print('running target')
target = decorate(target)```

At the end of either of these snippets, the `target` name refers to whatever `decorator(target)` returned.

They can replace a decorated function with another one

## When Python Executes Decorators

Decorators run right after the decorated function is defined

Function decorators run as soon as a module is imported, but decorated functions only run when they are explicitly invoked.

This demonstrates the difference between `import time` and `runtime`

Decorators are usually defined in a module sepearte to the functions they will decorate.

## Using decorators to enhance code

A registration decorator is one that adds the decorated function to some collections. This is useful for keeping track of all functions in a list for example.

Most decorators change the decorated function using an inner function.

## Scope

Python assumes a variable assigned in the body of a function is local

## Use dis.dis(function) all the time to get to Python's internals

## Closures

A closure is a funciton that contains nonglobal variables, referenced in the body of the function, but not defined there.

It is a record, that stores a function, along with an environment. The environment maps each free variable of the function(vars used locally but defined in the enclosing scope), with the value/reference to which it was bound when the closure was created.

High order function for running average

In [ ]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [ ]:
avg = make_averager()
avg(10)

In [ ]:
avg(11)

In [ ]:
avg(12)

In [ ]:
avg.__closure__

## The nonlocal Declaration

Introduced in Python 3, it lets you flag a variable as free, even when assigned a new value within the function.

In [ ]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        # as += is the same as count = count + 1,
        # Python will error without the nonlocal flag
        # because it assumes count is assigned in the function
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
        
    return averager

Use repr() to return the string represention of an object

## Implementing a simple decorator

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [ ]:
factorial(5)

In [ ]:
factorial.__name__

Looked at the above cell!

The decorator redefined what the functimon name points to.
The `clocked` function is assigned to factorial. When factorial(n) is called, clocked(n) is executed.

This is what usually happens with a decorator. It replace the decorated function with a new function. this new function accepts the same arguemnts and usually returns what the decorated function should return, while doing some extra processing.

## Decorators in the Standard Library

Python has three decorators specificaly for methods: property, classmehtod, and statismethod

fucntools.wraps is a decorator used for building well behaved decorators.

## memoization with functools.lru_cache

__Memoization__: technique that saves the results of previous invocations of an expensive function, avoiding repeat computions on previously used functions.

LRU stands for Least Recently Used - the growth of the cache is limited by discarding the entries that have not been read for a while.

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [ ]:
fibonacci(6)

Using lru_cache, far fewer calls are made:

In [ ]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

Noticed how you can have multiple decorators. This stacks them, like composite functions.

So:

@functool_cache()
@clock
def func()
    ...
    
Is the same as 

func = functool_cache(clock(func)))

In [ ]:
fibonacci(20)

Comment the lru_cache() decorator, and compare the execution time with big ns!

functools.lru_cache(maxsize=128, typed=False)

maxszie is how many calls are stored. For optimal performance make it a power of 2.

typed is whether or not to store results of different types seperatly.

All arguments taken from the decorated function must be hashable, as lru_cache uses a dict.

## Generic Functions with Single Dispatch

If you have a set of very similar tasks, that can be executed in very similar looking functions, ideally that code is shared. However, in Python you don't have function overloading, so you can't create variatiosn of one function with different signatures.

One could make one grand function with extensive ifs and elses to handle different arguments, that then calls specialised functions. But this is not maintainable, and the coupling between it and the special functions would become too tight.

functools.singledispatch is a decorator that makes the decorated function a generic function - a group of functions to perform the same operation in different ways, depening on the type of the argument

In [ ]:
from functools import singledispatch
from collections import abc
import numbers
import html

# singledispatch marks the base function
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}<\p>'.format(content)
    
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

Each specialised function is then decorated with <base_function>.register(type)

Also notice how some of the specialised functions are made to hangle ABCs including numbers.Integral, instead of a concrete implementation list int(or abc.MutableSequence instead of list).

This allows your program compatability with more types, and to support future classes that may be subclasses of your ABCs.

Use this ensure that each unit of code follows a single, logical task - as it should always.

## Parameterized Decorators

When parsing a decorator, Python gives the decorated function as the first argument of the decorator.

To create your own decorator function with parameters, make a decorator factory, that takes those parameters and returns a decorator. This returned decorator is what ends up being applied to the decorated function.

In [ ]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'
            % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [ ]:
@register(active=False)
def f1():
    print('running f1()')

register, the decorator factory, build the decorate function with the given params, then returns it.

Use set.discard(x) to discard element x from a set in Python

Hardcore Pythonistas think decorators are better coded as classes implementing \__call__

The first language to have first class functions was Lisp